# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_pd = pd.read_csv("output.csv")
cities_pd.head()

,City,ID,Lat,Long,Max Temp,Humidity,Cloudiness,Wind,Country,Date
0,atuona,4020109,-9.8000,-139.0333,79.29,77,22,18.81,PF,Wed Jun 23 18:37:31 2021
1,plettenberg bay,964712,-34.0527,23.3716,55.27,83,100,4.59,ZA,Wed Jun 23 18:37:31 2021
2,bonavista,5905393,48.6499,-53.1147,64.40,99,99,15.73,CA,Wed Jun 23 18:37:32 2021
3,namatanai,2090021,-3.6667,152.4333,84.25,70,100,3.65,PG,Wed Jun 23 18:37:32 2021
4,broken hill,2173911,-31.9500,141.4333,55.11,58,2,16.11,AU,Wed Jun 23 18:37:32 2021


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(g_key)

In [4]:
locations = cities_pd[["Lat", "Long"]].astype(float)
city_humidity = cities_pd["Humidity"].astype(float) 

In [5]:
center_coord = (40.52,34.34)

fig = gmaps.figure(center=center_coord,zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=city_humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)



fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
cities_df = pd.DataFrame(cities_pd)
cities_df

,City,ID,Lat,Long,Max Temp,Humidity,Cloudiness,Wind,Country,Date
0,atuona,4020109,-9.8000,-139.0333,79.29,77,22,18.81,PF,Wed Jun 23 18:37:31 2021
1,plettenberg bay,964712,-34.0527,23.3716,55.27,83,100,4.59,ZA,Wed Jun 23 18:37:31 2021
2,bonavista,5905393,48.6499,-53.1147,64.40,99,99,15.73,CA,Wed Jun 23 18:37:32 2021
3,namatanai,2090021,-3.6667,152.4333,84.25,70,100,3.65,PG,Wed Jun 23 18:37:32 2021
4,broken hill,2173911,-31.9500,141.4333,55.11,58,2,16.11,AU,Wed Jun 23 18:37:32 2021
...,...,...,...,...,...,...,...,...,...,...
554,pemangkat,1631637,1.1667,108.9667,78.26,85,98,6.89,ID,Wed Jun 23 18:40:54 2021
555,bentiu,377472,9.2333,29.8333,72.09,81,38,5.30,SS,Wed Jun 23 18:40:55 2021
556,chanika,160592,-5.4167,38.0167,59.13,98,67,2.95,TZ,Wed Jun 23 18:40:55 2021
557,chuguyevka,2025286,44.1653,133.8633,79.27,42,11,3.87,RU,Wed Jun 23 18:40:56 2021


In [7]:
# Ideal weather conditions: 75-85F, Wind Speed 5-25mph, humidity 10-60%

cities_ideal = cities_df[(cities_df['Max Temp'] < 82) & 
            (cities_df['Wind'] >= 5) & (cities_df['Wind'] <= 25) & 
            (cities_df['Humidity'] >= 15) & (cities_df['Humidity'] <= 55)]

cities_ideal_df = pd.DataFrame(cities_ideal)
cities_ideal_df.dropna(inplace=True)
print(len(cities_ideal_df))

47


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
cities_ideal_df=cities_ideal_df.drop(columns=["Date","ID"])
cities_ideal_df["Hotel Name"] = ""

In [9]:
print(cities_ideal_df)

                    City      Lat      Long  Max Temp  Humidity  Cloudiness  \
40           yellowknife  62.4560 -114.3525     59.56        33          20   
68             kununurra -15.7667  128.7333     75.18        50          90   
70             saskylakh  71.9167  114.0833     55.49        39          14   
72             shelburne  44.0787  -80.2041     66.69        50          78   
89          port hedland -20.3167  118.5667     59.05        51          62   
112                dukat  62.5500  155.5500     57.87        37         100   
123              birjand  32.8663   59.2211     59.47        20           0   
158             cherskiy  68.7500  161.3000     66.36        27           5   
159                korla  41.7597   86.1469     75.49        30           0   
183            tilichiki  60.4667  166.1000     65.73        45          14   
185                macon  39.8667  -88.9668     78.55        44           1   
217            flin flon  54.7682 -101.8650     64.9

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "39.952583,-75.16522",  # philadelphia coords
    "rankby": "distance",
    "type": "restaurant",
    "key": gkey,

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
